# Palmer Penguins Modeling

Import the Palmer Penguins dataset and print out the first few rows.

Suppose we want to predict `bill_depth_mm` using the other variables in the dataset.

**Dummify** all variables that require this.

In [18]:
!pip install scikit-learn
!pip install palmerpenguins
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from palmerpenguins import load_penguins
from plotnine import *
pen = load_penguins()

In [19]:
pen.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [20]:
# Dummify categorical variables
pen = pd.get_dummies(pen, columns=['species', 'island', 'sex'], drop_first=True)

# Dummified dataset
pen.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year,species_Chinstrap,species_Gentoo,island_Dream,island_Torgersen,sex_male
0,39.1,18.7,181.0,3750.0,2007,False,False,False,True,True
1,39.5,17.4,186.0,3800.0,2007,False,False,False,True,False
2,40.3,18.0,195.0,3250.0,2007,False,False,False,True,False
3,NaN,NaN,NaN,NaN,2007,False,False,False,True,False
4,36.7,19.3,193.0,3450.0,2007,False,False,False,True,False


Let's use the other variables to predict `bill_depth_mm`. Prepare your data and fit the following models on a training dataset subset of the entire dataset:

* Four different models, each containing a different set of predictor variables

Create a plot like the right plot of Fig 1. in our `Model Validation` chapter with the training and test error plotted for each of your four models.

Which of your models was best?

In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Define predictor variables for each model
predictors_model1 = ['bill_length_mm', 'flipper_length_mm', 'body_mass_g', 'species_Chinstrap', 'species_Gentoo']
predictors_model2 = ['bill_length_mm', 'flipper_length_mm', 'body_mass_g']
predictors_model3 = ['bill_length_mm', 'species_Chinstrap', 'species_Gentoo', 'island_Dream', 'island_Torgersen', 'sex_male']
predictors_model4 = ['bill_length_mm', 'flipper_length_mm', 'body_mass_g', 'species_Chinstrap', 'species_Gentoo', 'island_Dream', 'island_Torgersen', 'sex_male']


# Prepare the data
X = pen.drop('bill_depth_mm', axis=1)
y = pen['bill_depth_mm']

# Impute missing values in y before splitting
imputer = SimpleImputer(strategy='mean') # Create an imputer instance
y = imputer.fit_transform(y.values.reshape(-1, 1)) # Fit and transform y
y = y.ravel() # Convert back to 1D array

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Function to fit and evaluate models
def fit_and_evaluate_model(predictors, X_train, y_train, X_test, y_test):
    model = Pipeline([('imputer', SimpleImputer(strategy='mean')), # Add imputation step
                      ('poly', PolynomialFeatures(degree=2)),
                      ('linear', LinearRegression())])
    model.fit(X_train[predictors], y_train)
    train_r2 = r2_score(y_train, model.predict(X_train[predictors]))
    test_r2 = r2_score(y_test, model.predict(X_test[predictors]))
    return train_r2, test_r2

# Fit and evaluate each model
results = []
for i, predictors in enumerate([predictors_model1, predictors_model2, predictors_model3, predictors_model4]):
    train_r2, test_r2 = fit_and_evaluate_model(predictors, X_train, y_train, X_test, y_test)
    results.append({'Model': i+1, 'Train R^2': train_r2, 'Test R^2': test_r2})

results_df = pd.DataFrame(results)
results_df

,Model,Train R^2,Test R^2
0,1,0.814396,0.820942
1,2,0.474911,0.457803
2,3,0.826321,0.824575
3,4,0.602921,0.519099


In [24]:
# Calculate MSE from R²
def r2_to_mse(r2, y_true):
    return np.mean((y_true - np.mean(y_true)) ** 2) * (1 - r2)

# Calculate MSE for each model
train_mse = [r2_to_mse(row['Train R^2'], y_train) for row in results]
test_mse = [r2_to_mse(row['Test R^2'], y_test) for row in results]

# Add MSE values to results_df
results_df['Train MSE'] = train_mse
results_df['Test MSE'] = test_mse

# Reshape the dataframe for easier plotting with plotnine
plot_data = pd.melt(results_df, id_vars='Model', value_vars=['Train MSE', 'Test MSE'],
                    var_name='Set', value_name='MSE')

# Plot with plotnine
from plotnine import ggplot, aes, geom_line, geom_point, labs, theme_minimal

plot = (ggplot(plot_data, aes(x='Model', y='MSE', color='Set'))
        + geom_line(aes(group='Set'))
        + geom_point(size=3)
        + labs(x='Model Complexity', y='Mean Squared Error', title='Training and Test MSE by Model')
        + theme_minimal())
print(plot)

# Determine the best model based on Test R^2
best_model = results_df.loc[results_df['Test R^2'].idxmax()]
print(f"The best model is Model {int(best_model['Model'])} with a Test R^2 of {best_model['Test R^2']:.4f}")

<ggplot: (640 x 480)>
The best model is Model 3 with a Test R^2 of 0.8246
